In [11]:
import requests
from bs4 import BeautifulSoup
import re

# 직업 게시판 URL을 찾는 함수
def get_job_board_url(job_name):
    main_url = "https://lostark.inven.co.kr/"
    target_base_url = "https://www.inven.co.kr"  # 링크의 베이스 URL
    
    # 메인 페이지에서 게시판 목록 페이지로 이동
    response = requests.get(main_url)
    response.raise_for_status()  # 요청 에러 확인
    
    # HTML 파싱
    soup = BeautifulSoup(response.text, "html.parser")
    
    # 입력받은 직업 이름으로 링크 검색
    job_link = None
    links = soup.find_all("a", href=True)
    for link in links:
        if job_name in link.text:
            job_link = link.get("href")
            break
    
    if job_link:
        # 링크가 절대 경로인지 확인하고, 상대 경로인 경우에만 base URL 추가
        if not job_link.startswith("http"):
            full_url = target_base_url + job_link
        else:
            full_url = job_link
        return full_url
    else:
        return None

# 이모티콘 포함 글 제목 개수와 전체 글 개수를 세는 함수
def count_emoji_titles(job_board_url, num_pages):
    emoji_pattern = re.compile(r'[●▅]')  # 이모티콘 패턴 정의
    emoji_count = 0
    total_count = 0  # 전체 글 개수 초기화
    
    for page in range(1, num_pages + 1):
        # 각 페이지의 URL 구성
        page_url = f"{job_board_url}?p={page}"
        response = requests.get(page_url)
        response.raise_for_status()  # 요청 에러 확인
        
        # HTML 파싱
        soup = BeautifulSoup(response.text, "html.parser")
        
        # 글 제목 요소 추출
        titles = soup.find_all("a", class_="subject-link")  # 게시글 제목에 해당하는 클래스 수정 필요 가능성
        
        # 각 제목에서 이모티콘이 포함된 글 제목 개수 확인 및 전체 글 개수 증가
        for title in titles:
            total_count += 1
            if emoji_pattern.search(title.text):  # 이모티콘이 포함된 제목인지 확인
                emoji_count += 1

    return emoji_count, total_count

# 사용자 입력
job_name = input("직업 이름을 입력하세요: ")
num_pages = int(input("크롤링할 페이지 수를 입력하세요: "))

# 게시판 URL 가져오기
job_board_url = get_job_board_url(job_name)

if job_board_url:
    # 이모티콘이 포함된 글 제목의 개수와 전체 글 개수 확인
    print(f"현재 검색한 직업은 {job_name}입니다")
    emoji_title_count, total_title_count = count_emoji_titles(job_board_url, num_pages)
    print(f"{num_pages} 페이지에서 전체 글 제목의 개수: {total_title_count}")
    print(f"{num_pages} 페이지에서 이모티콘이 포함된 글 제목의 개수: {emoji_title_count}")
else:
    print("해당 직업의 게시판을 찾을 수 없습니다.")


현재 검색한 직업은 호크아이입니다
10 페이지에서 전체 글 제목의 개수: 504
10 페이지에서 이모티콘이 포함된 글 제목의 개수: 267
